In [81]:
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import

import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter

import torch.cuda
import numpy as np

from qpth.qp import QPFunction

import matplotlib.pyplot as plt


class LSTMModel(nn.Module):
    '''
    nn.LSTM Parameters:
        input_size  – The number of expected features in the input x
        hidden_size – The number of features in the hidden state h
        num_layers  – Number of recurrent layers.

    Inputs: input, (h_0, c_0)
        input (seq_len, batch, input_size)
        h_0 (num_layers * num_directions, batch, hidden_size)
        c_0 (num_layers * num_directions, batch, hidden_size)

    Outputs: output, (h_n, c_n)
        output (seq_len, batch, hidden_size * num_directions)
        h_n (num_layers * num_directions, batch, hidden_size)
        c_n (num_layers * num_directions, batch, hidden_size):
    '''
    def __init__(self, inputSize, nHidden, batchSize, noutputs=3, numLayers=2):
        super(LSTMModel, self).__init__()
        
        self.cost = nn.MSELoss(size_average=False)
        self.noutputs = noutputs
        self.num_layers = numLayers
        self.inputSize = inputSize
        self.nHidden = nHidden
        self.batchSize = batchSize
        self.noutputs = noutputs
        
        #define recurrent and linear layers
        self.lstm1  = nn.LSTM(inputSize,nHidden[0],num_layers=numLayers)
        self.lstm2  = nn.LSTM(nHidden[0],nHidden[1],num_layers=numLayers)
        self.lstm3  = nn.LSTM(nHidden[1],nHidden[2],num_layers=numLayers)
        self.fc     = nn.Linear(nHidden[2], noutputs)
        self.drop   = nn.Dropout(0.3)

    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[0])) 
        c0 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[0]))        
        # Forward propagate RNN layer 1
        out, _ = self.lstm1(x, (h0, c0)) 
        out = self.drop(out)
        
        # Set hidden layer 2 states 
        h1 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[1])) 
        c1 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[1]))        
        # Forward propagate RNN layer 2
        out, _ = self.lstm2(out, (h1, c1))  
        
        # Set hidden layer 3 states 
        h2 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[2])) 
        c2 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[2]))        
        # Forward propagate RNN layer 2
        out, _ = self.lstm3(out, (h2, c2)) 
        out = self.drop(out) 
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

#hyperparams
inputSize = 9
nHidden   = [9,6,6]
numLayers = 2
sequence_length = 9
num_epochs = 500
noutputs = 3
batchSize = 1

lstm = LSTMModel(inputSize, nHidden, batchSize, noutputs, numLayers)
#lstm.cuda()

# Loss and Optimizer
criterion = nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(lstm.parameters(), lr=0.1)

#images and labels
for epoch in range(num_epochs):
    inputs = Variable(torch.Tensor(5, 1, 9))
    labels = Variable(torch.Tensor(5, 3))

    # Forward + Backward + Optimize
    optimizer.zero_grad()
    outputs = lstm(inputs)
    loss    = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (epoch % 10) == 0:
        print ('Epoch [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, loss.data[0]))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch [1/500], Loss: inf
Epoch [11/500], Loss: nan
Epoch [21/500], Loss: nan
Epoch [31/500], Loss: nan
Epoch [41/500], Loss: nan
Epoch [51/500], Loss: nan
Epoch [61/500], Loss: nan
Epoch [71/500], Loss: nan
Epoch [81/500], Loss: nan
Epoch [91/500], Loss: nan
Epoch [101/500], Loss: nan
Epoch [111/500], Loss: nan
Epoch [121/500], Loss: nan
Epoch [131/500], Loss: nan
Epoch [141/500], Loss: nan
Epoch [151/500], Loss: nan
Epoch [161/500], Loss: nan
Epoch [171/500], Loss: nan
Epoch [181/500], Loss: nan
Epoch [191/500], Loss: nan
Epoch [201/500], Loss: nan
Epoch [211/500], Loss: nan
Epoch [221/500], Loss: nan
Epoch [231/500], Loss: nan
Epoch [241/500], Loss: nan
Epoch [251/500], Loss: nan
Epoch [261/500], Loss: nan
Epoch [271/500], Loss: nan
Epoch [281/500], Loss: nan
Epoch [291/500], Loss: nan
Epoch [301/500], Loss: nan
Epoch [311/500], Loss: nan
Epoch [321/500], Loss: nan
Epoch [331/500], Loss: nan
Epoch

In [44]:
'''
nn.LSTM Parameters:
    input_size  – The number of expected features in the input x
    hidden_size – The number of features in the hidden state h
    num_layers  – Number of recurrent layers.
    
Inputs: input, (h_0, c_0)
    input (seq_len, batch, input_size)
    h_0 (num_layers * num_directions, batch, hidden_size)
    c_0 (num_layers * num_directions, batch, hidden_size)

Outputs: output, (h_n, c_n)
    output (seq_len, batch, hidden_size * num_directions)
    h_n (num_layers * num_directions, batch, hidden_size)
    c_n (num_layers * num_directions, batch, hidden_size):
'''
rnnp = nn.LSTM(9, 9, 2)
inputp = Variable(torch.randn(5,1, 9))
h0p = Variable(torch.randn(2, 1, 9))
c0p = Variable(torch.randn(2, 1, 9))
outputp, hnp = rnnp(inputp, (h0p, c0p))
print('output, hn', outputp, hnp)

('output, hn', Variable containing:
(0 ,.,.) = 
  0.0594 -0.0667  0.1891 -0.1240  0.2621 -0.1618  0.1374  0.2346  0.1203

(1 ,.,.) = 
  0.0041 -0.0961  0.0826 -0.1564  0.1250 -0.2096 -0.0419  0.2256  0.0997

(2 ,.,.) = 
 -0.0424 -0.0660  0.0180 -0.0556  0.0714 -0.2139 -0.1316  0.1229  0.0921

(3 ,.,.) = 
 -0.0292 -0.0185  0.0154  0.0144  0.0572 -0.1743 -0.1293  0.0378  0.0829

(4 ,.,.) = 
  0.0014 -0.0254 -0.0038  0.0402  0.0844 -0.1310 -0.1433  0.0125  0.1028
[torch.FloatTensor of size 5x1x9]
, (Variable containing:
(0 ,.,.) = 
 -0.1835 -0.1025  0.1177  0.0033 -0.0459  0.1522  0.1153 -0.0820  0.1727

(1 ,.,.) = 
  0.0014 -0.0254 -0.0038  0.0402  0.0844 -0.1310 -0.1433  0.0125  0.1028
[torch.FloatTensor of size 2x1x9]
, Variable containing:
(0 ,.,.) = 
 -0.4481 -0.1853  0.2029  0.0166 -0.1076  0.4126  0.2311 -0.3411  0.3912

(1 ,.,.) = 
  0.0026 -0.0725 -0.0071  0.0987  0.1633 -0.2757 -0.4131  0.0285  0.2453
[torch.FloatTensor of size 2x1x9]
))


In [55]:
#inputs
print inputp

Variable containing:
(0 ,.,.) = 
 -0.5428  1.4186  1.8866  0.8649  0.6995  0.3084  0.9364 -0.2720  1.5090

(1 ,.,.) = 
  0.5815  0.3990  1.5051 -0.1367 -0.7284 -0.8908  0.4778  0.1386 -0.4370

(2 ,.,.) = 
 -0.6140 -0.2190  1.1750  1.1742  0.9553 -1.0940  1.0156  0.4339  0.3522

(3 ,.,.) = 
 -0.2989  0.3756 -0.5670  0.8080 -1.9982  0.9758  2.0426  2.8202 -0.4405

(4 ,.,.) = 
  2.0539 -0.1440 -0.2703 -1.1998  0.0871  0.3628  0.2336  2.0507  0.9338
[torch.FloatTensor of size 5x1x9]



In [10]:
import torch

In [25]:
x.cpu()
x.unsqueeze(0).expand(*([5]+list(x.size())))


(0 ,.,.) = 
  5.5608e+31  4.5883e-41  5.5608e+31  4.5883e-41  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  1.5695e-43

(1 ,.,.) = 
  5.5608e+31  4.5883e-41  5.5608e+31  4.5883e-41  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  1.5695e-43

(2 ,.,.) = 
  5.5608e+31  4.5883e-41  5.5608e+31  4.5883e-41  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  1.5695e-43

(3 ,.,.) = 
  5.5608e+31  4.5883e-41  5.5608e

In [30]:
x = torch.Tensor(5,5)
print x.ndimension()
x.mean(0).squeeze(0)

2



 1.1122e+31
 5.1643e+23
 1.0329e+24
 5.1643e+23
 1.0329e+24
[torch.FloatTensor of size 5]